In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import cv2
import numpy as np
import math

vid = cv2.VideoCapture(2)
white_image = np.ones((1080, 1920, 3), np.uint8) * 256

while True:
    whiteboard = np.ones((1080, 1920, 3), np.uint8) * 256
    ret, image = vid.read()
    y = image.shape[0]
    x = image.shape[1]
    halfx = int(x/2)
    yless = y - 1

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lower_value_threshold = 0
    upper_value_threshold = 150

    lower_hue_threshold = 30
    upper_hue_threshold = 90

    value_mask = cv2.inRange(v, lower_value_threshold, upper_value_threshold)
    hue_mask = cv2.inRange(h, lower_hue_threshold, upper_hue_threshold)
    composite_mask = cv2.bitwise_and(value_mask, hue_mask)
    segmented_image = cv2.bitwise_and(image, image, mask=composite_mask)

    img_gry = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(img_gry)
    img_blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    ret, img_Otsubin = cv2.threshold(img_gry, 10, 255, cv2.THRESH_BINARY)
    imagetocanny = img_Otsubin

    edges = cv2.Canny(image=imagetocanny, threshold1=200, threshold2=200)
    line_image = np.copy(img_Otsubin) * 0

    lines = cv2.HoughLinesP(edges, 1, np.pi/100, 15, np.array([]), 100, 30)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 5)
            cv2.line(whiteboard, (x1, y1), (x2, y2), (255, 0, 0), 5)

    result = []
    y1 = y - 1
    x1 = halfx
    while x1 > 0:
        pixel_color = whiteboard[y1, x1]
        if all(pixel_color == [255, 0, 0]):
            break
        y1 -= 1
        x1 -= 1
    cv2.line(image, (x1, y1), (halfx, y), (0, 255, 0), 5)
    result.append(int(math.hypot(halfx - x1, yless - y1)))

    y2 = yless
    x2 = halfx
    while y2 > 0:
        pixel_color = whiteboard[y2, x2]
        if all(pixel_color == [255, 0, 0]):
            break
        y2 -= 1
    cv2.line(image, (halfx, y2), (halfx, y), (0, 255, 0), 5)
    result.append(int(math.hypot(halfx - x2, 719 - y2)))

    y3 = yless
    x3 = halfx
    while x3 < x:
        pixel_color = whiteboard[y3, x3]
        if all(pixel_color == [255, 0, 0]):
            break
        y3 -= 1
        x3 += 1
    cv2.line(image, (x3, y3), (halfx, y), (0, 255, 0), 5)
    result.append(int(math.hypot(halfx - x3, 719 - y3)))
    print(result)

    cv2.imshow('frame', image)
    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

white_image = np.ones((1080, 1920, 3), np.uint8) * 256 
plt.imshow(cv2.cvtColor(white_image, cv2.COLOR_BGR2RGB))